In [1]:
import numpy as np
from sklearn.utils import shuffle
import math
import time
import matplotlib.pyplot as plt

In [2]:
path1 = "/Users/januaryshen/Dropbox/S19 - CSE 547/hw4/q1/data/features.txt"
path2 = "/Users/januaryshen/Dropbox/S19 - CSE 547/hw4/q1/data/target.txt"
x = np.matrix(np.loadtxt(path1, delimiter = ','))
y = np.loadtxt(path2, delimiter = ',')

In [3]:
def get_v_determinant(w, x, y, b):
    return(np.multiply((x.dot(w)+ b), y))

def leave_only_less_than_one(v):
    v = 1 - v
    v[v < 0] = 0
    return(v)   

def get_cost(w, x, y, b, C):
    c_w = 1/2*pow(np.linalg.norm(w),2)
    c_det = C * leave_only_less_than_one(get_v_determinant(w, x, y, b)).sum()
    return(c_w + c_det)

def cost_change(cost_prior, cost_after):
    return(abs(cost_prior-cost_after)*100/cost_prior)

def update_w(v_det, x, y, j):
    w_change = 0
    for i in range(v_det.shape[1]):
        if v_det[0, i] < 1:
            w_change += -y[i]*x[i, j]
    return(w_change)

# def update_w(w, x, y, b):
#     filter_v = get_v_determinant(w, x, y, b)
#     target_m = np.array(np.multiply(x, y.reshape(x.shape[0],1)))
#     target_m[filter_v >= 1] = 0

def update_b(v_det, y):
    b_change = 0
    for i in range(v_det.shape[1]):
        if v_det[0, i] < 1:
            b_change += -y[i]
    return(b_change)

In [ ]:
filter_v = get_v_determinant(w, x, y, b)
target_m = np.multiply(x, y.reshape(6414,1))
#target_m[filter_v >= 1] = 0

# for i in range(target_m.shape[1]):
#     filter_m[:, i] = filter_v[filter_v >= 1]
# target_m[filter_m == True] = 0

a = filter_v[[0]]
a.shape

In [ ]:
temp_m = np.matrix([[2,4], [6,8]])
temp_f = np.array([3,7])
temp_f.shape
#temp_m[temp_f > 5] = 100

In [ ]:
(x.dot(w)+ b).shape

In [ ]:
# settings for BGD
eta_BGD = 3*pow(10, -7)
epsilon_BGD = 0.25
w = np.zeros(x.shape[1])
#w = np.zeros()
b = np.zeros(1)
C = 100
k = 0
change_BGD = 1 # random number that's bigger than epsilon
new_w = np.zeros(x.shape[1])

cost_prior = get_cost(w, x, y, b, C)
cost_list_BGD = [(0, cost_prior)]

In [ ]:
# 1. Batch Gradient Descent (BGD)

startT = time.time()
while change_BGD > epsilon_BGD:
    
    v_det = get_v_determinant(w, x, y, b)

    new_w = w - eta_BGD * (w + C * update_w(w, x, y, b))
    
    b = b - eta_BGD * C * update_b(v_det, y)
    w = new_w
    k = k + 1
    
    cost_after = get_cost(w, x, y, b, C)
    cost_list_BGD.append((k, cost_after))
    change_BGD = cost_change(cost_prior, cost_after)
    cost_prior = cost_after

endT = time.time()
print(k, endT - startT)

In [ ]:
# 2. Stochastic Gradient Descent (SGD) 500-3000
x, y = shuffle(x, y, random_state=0)

eta_SGD = 0.0001
epsilon_SGD = 0.001

w = np.repeat(0, x.shape[1])
b = 0
k = 0
i = 1
C = 100
new_w = w
last_change = 0 # set the initial change as 0
change_SGD = 100 # random as long as it's bigger than eta
cost_list_SGD = list()

startT = time.time()
while change_SGD > epsilon_SGD:
    
    cost_prior = get_cost(w, x, y, b, C)
    scalar_det = get_determinant(w, x[i-1], y[i-1], b).sum()
    
    if scalar_det >= 1:
        w_temp = 0
        b_temp = 0
    else:
        w_temp = np.array(-y[i-1] * x[i-1, :])[0]
        b_temp = -y[i-1]
        
    new_w = w - eta_SGD * (w + (C * w_temp))
    w = new_w
    b = b - eta_SGD * C * b_temp
    
    if k == 0:
        cost_prior = 100
    
    cost_after = get_cost(w, x, y, b, C)
    change_SGD = 0.5*last_change + 0.5*cost_change(cost_prior, cost_after)
    last_change = change_SGD
    
    cost_list_SGD.append((k, cost_after))
    
    k += 1
    i = (i % x.shape[0]) + 1

endT = time.time()
print(k, endT-startT)

In [4]:
# 3. Mini-Batch Gradient Descent (MBGD)

x, y = shuffle(x, y, random_state=0)

k = 0
epsilon_MBGD = 0.01
eta_MBGD = pow(10, -5)
w = np.repeat(0, x.shape[1])
new_w = w
b = 0
C = 100
B = 20
l = 0

change_MBGD = 1 # random number as long as it's bigger than epsilon
last_change = 0
cost_list_MBGD = list()

startT = time.time()
while change_MBGD > epsilon_MBGD:
    
    cost_prior = get_cost(w, x, y, b, C)
    start = l*B + 1
    end = min(x.shape[0], (l+1)*B)
    
    mb_det = get_v_determinant(w, x[start:end], y[start:end], b)
    
    for j in range(x.shape[1]):
        w_temp = 0
        b_temp = 0
        for t in range(mb_det.shape[1]):
            if mb_det[0,t] >= 1:
                w_temp += 0
                b_temp += 0
            else:
                w_temp += -y[start+t]*x[start+t,j]
                b_temp += -y[start+t]
        new_w[j] = w[j] - eta_MBGD * (w[j] + C * w_temp)
    
    w = new_w
    b = b - eta_MBGD * C * b_temp
    k += 1
    l = (l + 1) % math.ceil(x.shape[0]/B)
    
    if k == 0:
        last_change = 0
    
    cost_after = get_cost(w, x, y, b, C)
    change_MBGD = 0.5*last_change + 0.5*cost_change(cost_prior, cost_after)
    last_change = change_MBGD
    
    cost_list_MBGD.append((k, cost_after))

endT = time.time()
print(k, endT-startT)

KeyboardInterrupt: 

In [ ]:
plt.plot(*zip(cost_list_BGD, cost_list_SGD, cost_list_MBGD))
plt.show()

In [ ]:
plt.plot(*zip(*cost_list_MBGD))

In [ ]:
plt.plot(*zip(cost_list_BGD))
plt.show()

In [ ]:
cost_list_BGD

In [ ]:
plt.plot(*zip(*cost_list_MBGD))
plt.show()